# Project 1

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV, SelectKBest, r_regression
from sklearn.gaussian_process.kernels import Matern, RBF
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, IsolationForest, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA

import xgboost as xgb

import torch
from torch import nn
from torch.nn import Module, Linear, Dropout
from torch.nn.functional import tanh, softmax, mse_loss, relu, sigmoid
from torch.optim import Adam, SGD

## Load Data

In [2]:
# load and split data
data_X_train = pd.read_csv('Data/X_train.csv', header=0, index_col=0)
data_y_train = pd.read_csv('Data/y_train.csv', header=0, index_col=0)
data_X_test = pd.read_csv('Data/X_test.csv', header=0, index_col=0)

In [3]:
# data info
data_X_train.describe()
"""
Data Shape: 1212 x 832
Data Lost: a lot
data scale: large
"""

'\nData Shape: 1212 x 832\nData Lost: a lot\ndata scale: large\n'

In [4]:
# transfer data to numpy
X_train = data_X_train.to_numpy()
y_train = data_y_train.to_numpy()
X_test = data_X_test.to_numpy()

## Data Preprocessing

### 归一化

In [5]:
x_scalar = StandardScaler()
X_train = x_scalar.fit_transform(X_train)
X_test = x_scalar.transform(X_test)

# y_scalar = StandardScaler()
# y_train = y_scalar.fit_transform(y_train)

### 处理缺省值

In [6]:
# KNN Imputer
imputer = KNNImputer(n_neighbors=5)
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

### 特征选择

#### 删除变化过小的列

In [7]:
del_columns_id_all0 = np.where(X_train.sum(axis=0) == 0)
X_train = np.delete(X_train, del_columns_id_all0, axis=1)
X_test = np.delete(X_test, del_columns_id_all0, axis=1)

#### 皮尔森系数

In [8]:
cc = r_regression(X_train, y_train.ravel())
del_columns_id_pearson = np.where(np.absolute(cc) <= 1e-2) # 删除pearson系数小于0.01的特征
X_train = np.delete(X_train, del_columns_id_pearson, axis=1)
X_test = np.delete(X_test, del_columns_id_pearson, axis=1)

#### 1. 删减特征

In [ ]:
# RFECV
estimator = LinearRegression()
selector = RFECV(estimator, step=1, cv=5, scoring='r2')
selector.fit(X_train, y_train)
feature_ranks = selector.ranking_

In [7]:
# store the feature ranking
# pd.DataFrame(selector.ranking_).to_csv("Temp/feature_ranking.csv", header=False, index=False)
feature_ranks = pd.read_csv("Temp/feature_ranking.csv", header=None, index_col=None).to_numpy().reshape(-1)

In [8]:
# choose features (top 95%)
def select_features(x, rank, threshold=0.8):
    drop_feature_ids = np.where(rank > int(threshold * max(rank)))
    selected_x = np.delete(x, drop_feature_ids, axis=1)
    return selected_x

X_train = select_features(X_train, feature_ranks, 0.95)
X_test = select_features(X_test, feature_ranks, 0.95)

#### 保留特征

In [9]:
# 使用selectkbest方法选择特征
skb = SelectKBest(k=200)
X_train = skb.fit_transform(X_train,y_train.ravel())
X_test = skb.transform(X_test)
print(X_train.shape)

(1212, 200)


#### PCA

In [9]:
pca = PCA(n_components=50)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

### 噪声探测

In [10]:
"""
！！！该方法具有较高的不确定性，不能保证有效
思路：
    1. 以每一个实际y为单位，找出上下年龄差为5的数据
    2. 计算该数据到这一组数据的均值的距离
    3. 如果距离大于某一阈值，则删除该数据
"""
ERROR_COEFFICIENT = 2

deleted_sample_ids = []
errors = []
for row_i in range(X_train.shape[0]):
    X_row = X_train[row_i]
    y_row = y_train[row_i]

    sub_y_X = X_train[((y_train >= y_row - 5) & (y_train <= y_row + 5)).ravel()]
    if sub_y_X.shape[0] > 10:
        sub_y_X_mean = np.average(sub_y_X, axis=0)
        sub_y_X_std = np.std(sub_y_X, axis=0)
        sub_y_X_error = (X_row > sub_y_X_mean + ERROR_COEFFICIENT * sub_y_X_std) | (X_row < sub_y_X_mean - ERROR_COEFFICIENT * sub_y_X_std)
        errors.append(sub_y_X_error.sum())
        if sub_y_X_error.sum() > X_train.shape[1] * 0.15:
            deleted_sample_ids.append(row_i)
    else:
        continue
X_train = np.delete(X_train, deleted_sample_ids, axis=0)
y_train = np.delete(y_train, deleted_sample_ids, axis=0)
print(X_train.shape)
print(y_train.shape)

(1136, 200)
(1136, 1)


In [10]:
# 另一种方法 用四分位数解决outlier的问题
IQR_threshold = 15
IQR_efficient = 2


X_train = pd.DataFrame(X_train)
q1 = X_train.quantile(0.25)
q3 = X_train.quantile(0.75)
IQR = q3 - q1
mask = ((X_train < (q1 - IQR_efficient * IQR)) | (X_train > (q3 + IQR_efficient * IQR)))

# outliers = X_train[mask]
# outliers
mask = mask.sum(axis=1) > IQR_threshold
outliers = X_train[mask]
X_train = X_train[~mask].to_numpy()
y_train = y_train[~mask]
print(X_train.shape)
print(y_train.shape)

(1169, 200)
(1169, 1)


## Model Selection

### Linear Regression

In [15]:
# LR
fold_num = 5

kf = KFold(n_splits=fold_num)
fold_score = 0
for i, (train_ids, valid_ids) in enumerate(kf.split(X_train)):
    # split validation data
    fold_X_train = X_train[train_ids]
    fold_y_train = y_train[train_ids]
    fold_X_valid = X_train[valid_ids]
    fold_y_valid = y_train[valid_ids]

    # train model
    model = LinearRegression()
    model.fit(fold_X_train, fold_y_train)
    fold_y_pred = model.predict(fold_X_valid)

    # calculate score
    fold_score += r2_score(fold_y_valid, fold_y_pred)
fold_score /= fold_num
print(fold_score)

-2.890191269931073


### Kernel Ridge Regression

In [20]:
# Ridge Regression
fold_num = 5

kf = KFold(n_splits=fold_num)
fold_score = 0
for i, (train_ids, valid_ids) in enumerate(kf.split(X_train)):
    # split validation data
    fold_X_train = X_train[train_ids]
    fold_y_train = y_train[train_ids]
    fold_X_valid = X_train[valid_ids]
    fold_y_valid = y_train[valid_ids]

    # train model
    model = KernelRidge(kernel="rbf")
    model.fit(fold_X_train, fold_y_train)
    fold_y_pred = model.predict(fold_X_valid)

    # calculate score
    fold_score += r2_score(fold_y_valid, fold_y_pred)
fold_score /= fold_num
print(fold_score)

0.019787009919412422


### Gaussian Process Regressor

In [32]:
# Gaussian Process Regressor (Matern)
fold_num = 5

kf = KFold(n_splits=fold_num)
fold_scores = []
for i, (train_ids, valid_ids) in enumerate(kf.split(X_train)):
    # split validation data
    fold_X_train = X_train[train_ids]
    fold_y_train = y_train[train_ids]
    fold_X_valid = X_train[valid_ids]
    fold_y_valid = y_train[valid_ids]

    # train model
    model = GaussianProcessRegressor(kernel=Matern(nu=0.5, length_scale=1), random_state=0)
    model.fit(fold_X_train, fold_y_train)
    fold_y_pred = model.predict(fold_X_valid)

    # calculate score
    fold_scores.append(r2_score(fold_y_valid, fold_y_pred))
fold_score = np.average(fold_scores)
print(fold_score)

0.3749308857165471


In [38]:
# Gaussian Process Regressor (RBF)
fold_num = 5

kf = KFold(n_splits=fold_num)
fold_scores = []
for i, (train_ids, valid_ids) in enumerate(kf.split(X_train)):
    # split validation data
    fold_X_train = X_train[train_ids]
    fold_y_train = y_train[train_ids]
    fold_X_valid = X_train[valid_ids]
    fold_y_valid = y_train[valid_ids]

    # train model
    model = GaussianProcessRegressor(kernel=RBF(length_scale=10), random_state=0)
    model.fit(fold_X_train, fold_y_train)
    fold_y_pred = model.predict(fold_X_valid)

    # calculate score
    fold_scores.append(r2_score(fold_y_valid, fold_y_pred))
fold_score = np.average(fold_scores)
print(fold_score)

-0.01484695627455086


### SVR

In [47]:
# SVR
fold_num = 5

kf = KFold(n_splits=fold_num)
fold_scores = []
for i, (train_ids, valid_ids) in enumerate(kf.split(X_train)):
    # split validation data
    fold_X_train = X_train[train_ids]
    fold_y_train = y_train[train_ids]
    fold_X_valid = X_train[valid_ids]
    fold_y_valid = y_train[valid_ids]

    # train model
    model = SVR(kernel="rbf")
    model.fit(fold_X_train, fold_y_train.ravel())
    fold_y_pred = model.predict(fold_X_valid)

    # calculate score
    fold_scores.append(r2_score(fold_y_valid, fold_y_pred))
fold_score = np.average(fold_scores)
print(fold_score)

0.27278034373557075


### Random Forest

In [62]:
# Random Forest Regressor
fold_num = 5

kf = KFold(n_splits=fold_num)
fold_scores = []
for i, (train_ids, valid_ids) in enumerate(kf.split(X_train)):
    # split validation data
    fold_X_train = X_train[train_ids]
    fold_y_train = y_train[train_ids]
    fold_X_valid = X_train[valid_ids]
    fold_y_valid = y_train[valid_ids]

    # train model
    model = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=0, criterion="squared_error")
    model.fit(fold_X_train, fold_y_train.ravel())
    fold_y_pred = model.predict(fold_X_valid)

    # calculate score
    fold_scores.append(r2_score(fold_y_valid, fold_y_pred))
fold_score = np.average(fold_scores)
print(fold_score)

0.45761414755344837


In [67]:
# Isolation Forest Regressor
fold_num = 5

kf = KFold(n_splits=fold_num)
fold_scores = []
for i, (train_ids, valid_ids) in enumerate(kf.split(X_train)):
    # split validation data
    fold_X_train = X_train[train_ids]
    fold_y_train = y_train[train_ids]
    fold_X_valid = X_train[valid_ids]
    fold_y_valid = y_train[valid_ids]

    # train model
    model = IsolationForest(n_estimators=100, max_features=0.6, random_state=0)
    model.fit(fold_X_train, fold_y_train)
    fold_y_pred = model.predict(fold_X_valid)

    # calculate score
    fold_scores.append(r2_score(fold_y_valid, fold_y_pred))
fold_score = np.average(fold_scores)
print(fold_score)

-50.88352200279015


In [11]:
# Gradient Boosting Regressor
fold_num = 5

kf = KFold(n_splits=fold_num)
fold_scores = []
for i, (train_ids, valid_ids) in enumerate(kf.split(X_train)):
    # split validation data
    fold_X_train = X_train[train_ids]
    fold_y_train = y_train[train_ids]
    fold_X_valid = X_train[valid_ids]
    fold_y_valid = y_train[valid_ids]

    # train model
    model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1)
    model.fit(fold_X_train, fold_y_train.ravel())
    fold_y_pred = np.round(model.predict(fold_X_valid))

    # calculate score
    fold_scores.append(r2_score(fold_y_valid, fold_y_pred))
fold_score = np.average(fold_scores)
print(fold_score)

0.6455286476185035


In [72]:
# Adaboost Regressor
fold_num = 5

kf = KFold(n_splits=fold_num)
fold_scores = []
for i, (train_ids, valid_ids) in enumerate(kf.split(X_train)):
    # split validation data
    fold_X_train = X_train[train_ids]
    fold_y_train = y_train[train_ids]
    fold_X_valid = X_train[valid_ids]
    fold_y_valid = y_train[valid_ids]

    # train model
    model = AdaBoostRegressor(n_estimators=100, learning_rate=0.1, loss="square")
    model.fit(fold_X_train, fold_y_train.ravel())
    fold_y_pred = model.predict(fold_X_valid)

    # calculate score
    fold_scores.append(r2_score(fold_y_valid, fold_y_pred))
fold_score = np.average(fold_scores)
print(fold_score)

0.4110280800135076


### XGBoost

In [11]:
# XGBoost
fold_num = 5

kf = KFold(n_splits=fold_num)
fold_scores = []
for i, (train_ids, valid_ids) in enumerate(kf.split(X_train)):
    # split validation data
    fold_X_train = X_train[train_ids]
    fold_y_train = y_train[train_ids]
    fold_X_valid = X_train[valid_ids]
    fold_y_valid = y_train[valid_ids]

    # train model
    model = xgb.XGBRegressor(n_estimators=150, max_depth=5, learning_rate=0.11, n_jobs=20)
    model.fit(fold_X_train, fold_y_train.ravel())
    fold_y_pred = np.round(model.predict(fold_X_valid))

    # calculate score
    fold_scores.append(r2_score(fold_y_valid, fold_y_pred))
fold_score = np.average(fold_scores)
print(fold_score)

0.6628370028029413


### Neural Network

In [30]:
class NNModel(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.nn1 = Linear(200, 256)
        # self.dropout1 = Dropout(0.5)
        self.nn2 = Linear(256, 64)
        # self.dropout2 = Dropout(0.5)
        self.nn3 = Linear(64, 1)
        
    def forward(self, x):
        output = sigmoid(self.nn1(x))
        # output = self.dropout1(output)
        output = sigmoid(self.nn2(output))
        # output = self.dropout2(output)
        output = self.nn3(output)
        return output

In [32]:
# Neural Network
fold_num = 5

kf = KFold(n_splits=fold_num)
fold_scores = []
for i, (train_ids, valid_ids) in enumerate(kf.split(X_train)):
    # split validation data
    fold_X_train = torch.from_numpy(X_train[train_ids]).float()
    fold_y_train = torch.from_numpy(y_train[train_ids]).float()
    fold_X_valid = torch.from_numpy(X_train[valid_ids]).float()
    fold_y_valid = torch.from_numpy(y_train[valid_ids]).float()

    # train model
    model = NNModel()
    optimizer = SGD(model.parameters(), lr=0.015, momentum=0.9)
    epoch_num = 150
    for epoch in range(epoch_num):
        optimizer.zero_grad()
        output = model(fold_X_train)
        loss = mse_loss(output, fold_y_train)

        loss.backward()
        optimizer.step()
        valid_loss = mse_loss(model(fold_X_valid), fold_y_valid)
        print("The epoch is {}, the loss is {:.03f}, the loss in validation data is {:.03f}".format(epoch, loss, valid_loss))

    fold_y_pred = model(fold_X_valid).detach().numpy()

    # calculate score
    fold_scores.append(r2_score(fold_y_valid, fold_y_pred))
fold_score = np.average(fold_scores)
print(fold_score)

The epoch is 0, the loss is 5015.557, the loss in validation data is 1451.557
The epoch is 1, the loss is 1441.213, the loss in validation data is 4181.361
The epoch is 2, the loss is 4220.127, the loss in validation data is 1434.339
The epoch is 3, the loss is 1460.927, the loss in validation data is 1288.183
The epoch is 4, the loss is 1279.185, the loss in validation data is 2626.787
The epoch is 5, the loss is 2609.954, the loss in validation data is 121.457
The epoch is 6, the loss is 132.830, the loss in validation data is 2393.577
The epoch is 7, the loss is 2425.821, the loss in validation data is 394.511
The epoch is 8, the loss is 411.596, the loss in validation data is 1305.913
The epoch is 9, the loss is 1296.785, the loss in validation data is 1196.299
The epoch is 10, the loss is 1187.971, the loss in validation data is 301.476
The epoch is 11, the loss is 317.130, the loss in validation data is 1445.790
The epoch is 12, the loss is 1472.476, the loss in validation data i

In [28]:
pd.DataFrame([(fold_y_valid.detach().numpy() * 100).ravel(), (fold_y_pred * 100).ravel()])

,0,1,2,3,4,5,6,7,8,9,...,217,218,219,220,221,222,223,224,225,226
0,88.00000,77.0000,66.000000,63.000000,70.000000,49.000000,77.000000,85.00000,63.000000,52.999996,...,77.000000,76.000000,80.000000,56.000000,69.000000,75.000000,86.00000,68.000000,71.000000,52.999996
1,70.98037,70.7006,74.506813,73.756241,68.706367,72.723038,75.560249,68.03923,71.611771,74.690208,...,76.683571,76.612625,72.068871,70.635918,65.719597,69.181099,69.32547,69.093559,72.001694,75.453476


## 预测

In [12]:
model = xgb.XGBRegressor(n_estimators=200, max_depth=7, learning_rate=0.1, n_jobs=20)
model.fit(X_train, y_train.ravel())
y_pred = np.round(model.predict(X_test))
y_pred_df = pd.DataFrame(y_pred, columns=["y"], index=data_X_test.index).reset_index()
y_pred_df["id"] = y_pred_df["id"].astype(int)
y_pred_df

,id,y
0,0,61.0
1,1,76.0
2,2,70.0
3,3,70.0
4,4,72.0
...,...,...
771,771,66.0
772,772,76.0
773,773,75.0
774,774,71.0


In [13]:
y_pred_df.to_csv("result_1.csv", index=False)